Download, unpack, and import into database the 2010 Census Block Shapefiles
===========================================================================

In [1]:
import csv, glob, json, os, re, shutil
import subprocess, sys, threading, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

Census 2010 subdivisions, for U.S. only:
    
    ~50 states
    ~73K census tracts (approx 4000 people)
    ~218K block groups (approx 1500 people)
    ~11M blocks

How to set up postgres
======================
    
    sudo -u postgres createuser --superuser $USER
    sudo -u postgres createdb $USER
    touch ~/.psql_history

In [7]:
!psql -c 'CREATE DATABASE timelapse;'
psql('CREATE EXTENSION IF NOT EXISTS postgis;')

ERROR:  database "timelapse" already exists
CREATE EXTENSION IF NOT EXISTS postgis;
Finished execution: CREATE EXTENSION

NOTICE:  extension "postgis" already exists, skipping



In [8]:
capture_dir = "capture/tabblock_2000"
try:
    os.makedirs(capture_dir)
except:
    pass

Download 2000 Census Block Shapefiles
-------------------------------------

Manually go to http://www.nhgis.org/, select 2000 census block shapefiles, all 50 states, to download a 2GB zip.  Rename this zip to capture/tabblock_2000/nhgis_2000_shapefiles.zip.

Unzip 2000 Census Block Shapefiles
----------------------------------

In [2]:
psql('\d')

\d
Finished execution in 1.19702 secs: List of relations
 Schema |                Name                 |   Type   |  Owner   
--------+-------------------------------------+----------+----------
 public | geography_columns                   | view     | rsargent
 public | geometry_columns                    | view     | rsargent
 public | od_jt00_2011                        | table    | rsargent
 public | od_jt00_2011_gid_seq                | sequence | rsargent
 public | od_jt01_2011                        | table    | rsargent
 public | od_jt01_2011_gid_seq                | sequence | rsargent
 public | raster_columns                      | view     | rsargent
 public | raster_overviews                    | view     | rsargent
 public | spatial_ref_sys                     | table    | rsargent
 public | tiger2010_census2000_blocks         | table    | rsargent
 public | tiger2010_census2000_blocks_gid_seq | sequence | rsargent
 public | tiger2010_census2010_blocks         | table    

In [46]:
missing = 0
for state_name in state_names:
    if not os.path.exists(capture_dir + '/%s_block_2000.shp' % state_name.upper()):
        missing += 1
if missing > 0:
    print subprocess.check_output('unzip capture/tabblock_2000/nhgis_2000_shapefiles.zip -d capture/tabblock_2000', shell=True)
    for zipfile in glob.glob(capture_dir +'/*/*.zip'):
        print zipfile
        cmd = 'unzip -o %s -d %s' % (zipfile, capture_dir)
        print cmd
        print subprocess.check_output(cmd, shell=True)

Reproject shapefiles from ESRI conic to WGS84 lat/lon
-----------------------------------------------------

In [17]:
def reproject_shapefile(state_name):
    state_name = state_name.upper()
    src = 'capture/tabblock_2000/{state_name}_block_2000.shp'.format(state_name=state_name)
    dest = os.path.splitext(src)[0] + '_4326.shp'
    if os.path.exists(dest):
        sys.stdout.write('%s already exists\n' % dest)
        return
    cmd = 'ogr2ogr -f "ESRI Shapefile" {dest} {src} -t_srs EPSG:4326'.format(src=src, dest=dest)
    sys.stdout.write(state_name + ': ' + subprocess.check_output(cmd, shell=True) + '\n')

threads = []

for state_name in state_names:
    threads.append(threading.Thread(target=reproject_shapefile, args=(state_name,)))
    threads[-1].start()

for t in threads:
    t.join()

capture/tabblock_2000/AK_block_2000_4326.shp already exists
capture/tabblock_2000/AL_block_2000_4326.shp already exists
capture/tabblock_2000/AR_block_2000_4326.shp already exists
capture/tabblock_2000/AZ_block_2000_4326.shp already exists
capture/tabblock_2000/CA_block_2000_4326.shp already exists
capture/tabblock_2000/CO_block_2000_4326.shp already exists
capture/tabblock_2000/CT_block_2000_4326.shp already exists
capture/tabblock_2000/DC_block_2000_4326.shp already exists
capture/tabblock_2000/DE_block_2000_4326.shp already exists
capture/tabblock_2000/FL_block_2000_4326.shp already exists
capture/tabblock_2000/GA_block_2000_4326.shp already exists
capture/tabblock_2000/HI_block_2000_4326.shp already exists
capture/tabblock_2000/IA_block_2000_4326.shp already exists
capture/tabblock_2000/ID_block_2000_4326.shp already exists
capture/tabblock_2000/IL_block_2000_4326.shp already exists
capture/tabblock_2000/IN_block_2000_4326.shp already exists
capture/tabblock_2000/KS_block_2000_4326

Load 2000 census block shapefiles into psql
-------------------------------------------

In [19]:
# Warning:  only run this block once or you may end up with duplicate records in the database
# TODO: check if the database is already there and populated and if so, skip the work...

def load_shapes_into_db(state_name, prepare_only=False):
    command = 'shp2pgsql'
    if prepare_only:
        command += ' -p'   # prepare the tables only, don't load
        sys.stdout.write('Creating table tl_2000_tabblock schema using state %s\n' % state_name)
    else:
        command += ' -a'   # append to tables
        sys.stdout.write('Appending state %s to tl_2000_tabblock\n' % state_name)
    command += ' -s 4326' # Shapefiles are in EPSG:4326 -- WGS84 lat,lon
    command += ' capture/tabblock_2000/%s_block_2000_4326.shp tl_2000_tabblock' % (state_name.upper())
    command += ' | psql -q -d timelapse'
    print command
    out = subprocess.check_output(command, shell=True)
    sys.stdout.write('psql output: %s\n' % out.strip())
    
# Drop and recreate tl_2000_tabblock    
psql('DROP TABLE tl_2000_tabblock;')
load_shapes_into_db(state_names[0], prepare_only=True)

# Loading all states in parallel worked OK on 32-core 64GB earthdev2.  psql was i/o bound, but no paging occurred
# On a lesser machine it might be important to do only a few in parallel at a time
threads = []

for state_name in state_names:
    threads.append(threading.Thread(target=load_shapes_into_db, args=(state_name, False)))
    threads[-1].start()

for t in threads:
    t.join()

DROP TABLE tl_2000_tabblock;
Finished execution in 0.122959 secs: DROP TABLE
Creating table tl_2000_tabblock schema using state ak
shp2pgsql -p -s 4326 capture/tabblock_2000/AK_block_2000_4326.shp tl_2000_tabblock | psql -q -d timelapse
psql output: addgeometrycolumn                         
------------------------------------------------------------------
 public.tl_2000_tabblock.geom SRID:4326 TYPE:MULTIPOLYGON DIMS:2 
(1 row)
Appending state ak to tl_2000_tabblock
shp2pgsql -a -s 4326 capture/tabblock_2000/AK_block_2000_4326.shp tl_2000_tabblock | psql -q -d timelapse
Appending state al to tl_2000_tabblock
shp2pgsql -a -s 4326 capture/tabblock_2000/AL_block_2000_4326.shp tl_2000_tabblock | psql -q -d timelapse
Appending state ar to tl_2000_tabblock
shp2pgsql -a -s 4326 capture/tabblock_2000/AR_block_2000_4326.shp tl_2000_tabblock | psql -q -d timelapse
Appending state az to tl_2000_tabblock
shp2pgsql -a -s 4326 capture/tabblock_2000/AZ_block_2000_4326.shp tl_2000_tabblock | psql -q

In [20]:
# Add indices to census block shapefile table
psql('CREATE INDEX ON tl_2000_tabblock (stfid);')
psql('CREATE INDEX ON tl_2000_tabblock USING GIST (geom);')


CREATE INDEX ON tl_2000_tabblock (stfid);
Finished execution in 67.0839 secs: CREATE INDEX
CREATE INDEX ON tl_2000_tabblock USING GIST (geom);
Finished execution in 147.186 secs: CREATE INDEX


tl_2000_tabblock table structure
----------------------------------

stfid is the unique identifier:  contains state, county, tract ... down to block ID.

In [21]:
psql('\d tl_2000_tabblock')

\d tl_2000_tabblock
Finished execution in 0.23154 secs: Table "public.tl_2000_tabblock"
  Column   |            Type             |                           Modifiers                            
-----------+-----------------------------+----------------------------------------------------------------
 gid       | integer                     | not null default nextval('tl_2000_tabblock_gid_seq'::regclass)
 fipsstco  | character varying(5)        | 
 tract2000 | character varying(6)        | 
 block2000 | character varying(4)        | 
 stfid     | character varying(15)       | 
 gisjoin   | character varying(18)       | 
 gisjoin2  | character varying(17)       | 
 geom      | geometry(MultiPolygon,4326) | 
Indexes:
    "tl_2000_tabblock_pkey" PRIMARY KEY, btree (gid)
    "tl_2000_tabblock_geom_idx" gist (geom)
    "tl_2000_tabblock_stfid_idx" btree (stfid)


In [22]:
psql('SELECT COUNT(*) FROM tl_2000_tabblock')

SELECT COUNT(*) FROM tl_2000_tabblock
Finished execution in 2.28785 secs: count  
---------
 8192957
(1 row)


In [24]:
psql("SELECT COUNT(*) FROM tl_2000_tabblock where stfid='010010201001000'")

SELECT COUNT(*) FROM tl_2000_tabblock where stfid='010010201001000'
Finished execution in 0.114097 secs: count 
-------
     1
(1 row)


In [32]:
psql("SELECT ST_AREA(geography(geom)) FROM tl_2000_tabblock LIMIT 10")

SELECT ST_AREA(geography(geom)) FROM tl_2000_tabblock LIMIT 10
Finished execution in 0.180544 secs: st_area      
------------------
 2132156.80175638
 384042.045816898
 1518912.70490001
 1067739.41412272
 6269152.85128927
 26294.5077588786
 6107772.71861947
 24121.5357999052
 366370.607746869
  15937.774944067
(10 rows)


In [25]:
psql("SELECT SUM(ST_AREA(geom)) FROM tl_2000_tabblock")

SELECT SUM(ST_AREA(geom)) FROM tl_2000_tabblock
Finished execution in 6.84909 secs: sum       
-----------------
 1096.3241191703
(1 row)


In [33]:
psql("SELECT LEFT(fipsstco,2), SUM(ST_AREA(geography(geom))) FROM tl_2000_tabblock GROUP BY LEFT(fipsstco,2) ORDER BY LEFT(fipsstco,2)")

SELECT LEFT(fipsstco,2), SUM(ST_AREA(geography(geom))) FROM tl_2000_tabblock GROUP BY LEFT(fipsstco,2) ORDER BY LEFT(fipsstco,2)
Finished execution in 399.731 secs: left |       sum        
------+------------------
 01   | 133741976870.766
 02   | 1492131345765.83
 04   | 295256652069.614
 05   | 137733198120.475
 06   | 409391754951.963
 08   | 269601387315.455
 09   | 12872098456.6564
 10   | 5131438591.61849
 11   |  159396612.81157
 12   | 146829817366.733
 13   | 152212840848.887
 15   | 16661623311.4829
 16   | 216442049362.762
 17   | 145917640496.456
 18   | 93711486566.8748
 19   | 145741540184.612
 20   | 213096312534.122
 21   | 104659205342.592
 22   | 119385435431.421
 23   | 84140541254.6559
 24   | 25709866467.5244
 25   | 21013813344.2774
 26   |  150597710314.18
 27   | 218532345986.363
 28   | 123453257843.056
 29   | 180533551839.555
 30   | 380830590679.682
 31   | 200343837053.449
 32   | 286351583217.581
 33   |  24000085040.424
 34   | 19594242110.8089
 35   | 3